In [164]:
import os, json, glob, math
import tensorflow as tf
import numpy as np

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
# %matplotlib inline

import pandas as pd

from scipy import signal

In [133]:
import sys
sys.path.append("./pyusct/")
from rfdata import RFdata

In [134]:
dir_dataset = "/mnt/nas/PYUSCT_train/dataset004/"

In [168]:
input_dataset = sorted(glob.glob( os.path.join(dir_dataset, "input/*.npy")))
output_dataset = sorted(glob.glob( os.path.join(dir_dataset, "output/*.npy")))
df = pd.read_csv(os.path.join(dir_dataset, "list.csv"))

assert len(input_dataset) == len(output_dataset) == len(df)
print (len(df))

1000


### 教師データ入出力画像を保存

散乱体画像の強度幅をサンプルから取得

In [143]:
sct_mean = np.mean(rf.medium_sct.flatten())
sct_mean

14459.479153255174

R-t画像可視化用関数

In [120]:
def translate_circular(img):

    num_angle= img.shape[0]
    img_size = int(img.shape[1]*2/np.sqrt(2))

    img_ = np.zeros((img_size, img_size), dtype=img.dtype)

    for i in range(img_size):
        for j in range(img_size):
            i_ = i - img_size //2
            j_ = j - img_size //2

            d = int(np.linalg.norm([i_, j_]))
            if d < img.shape[1]:
                k = ( int(( math.atan2(i_, j_) + np.pi ) // (2*np.pi/num_angle)) )% num_angle

                img_[i, j] = img[k, d]
    
    return img_

In [165]:
df.columns

Index(['id', 'source', 'ix', 'iy', 'x', 'y'], dtype='object')

In [179]:
dir_save = os.path.join(dir_dataset, 'image')
if not os.path.exists(dir_save): os.makedirs(dir_save)
    
sourcepath = ""
rf = None

for i in range(len(df)):
# for i in range(5):
    
    input_sample = np.load(input_dataset[i])
    output_sample = np.load(output_dataset[i])
    
    src = df.at[i, 'source']
    ix = df.at[i, 'ix']
    iy = df.at[i, 'iy']
    
    print((i, iy, ix))
    if(sourcepath != src):
        sourcepath = src
        rf = RFdata(src)
        
    img = np.mean(input_sample, axis=0)
    
    _vmax = 0.005
    _vmin = np.power(0.005, 2)
#     _vmax = np.log(0.005)
#     _vmin = np.log(np.power(0.005, 2))
    
    plt.clf()
    
    fig = plt.figure(figsize=(20,20))
    
    plt.subplot(2,2,1)
    plt.imshow(translate_circular(img), interpolation='nearest', cmap='hot', vmin=_vmin, vmax=_vmax)
    plt.title('RF.mean(T) circular')
    
    plt.subplot(2,2,2)
    ts = np.mean(np.mean(input_sample, axis=0), axis=0)
    plt.plot(ts)
    plt.plot( np.array([len(ts)//2]*2), np.array([_vmin, ts[len(ts)//2]]), color='red')
    plt.ylim(_vmin, _vmax)
    plt.title('RF.mean(T,R)')

    plt.subplot(2,2,3)
    img_whole = np.copy(rf.medium_sct)
    linecolor = 2*np.max(img_whole)
    img_whole[iy-1:iy+2,:] = linecolor
    img_whole[:,ix-1:ix+2] = linecolor
    plt.imshow(img_whole, interpolation='nearest')
    plt.title('scattering magnitude')

    plt.subplot(2,2,4)
    plt.imshow(output_sample, interpolation='nearest', vmin=sct_min, vmax=sct_mean)
    plt.title('scattering magnitude')

    fig.savefig(os.path.join(dir_save, '{0:0>8}.png'.format(i)))
    
        

(0, 410, 642)


/root/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


(1, 354, 430)
(2, 527, 663)
(3, 287, 533)
(4, 367, 342)
(5, 333, 681)
(6, 432, 552)
(7, 655, 372)
(8, 299, 587)
(9, 454, 507)
(10, 523, 468)
(11, 587, 391)
(12, 479, 687)
(13, 736, 453)
(14, 697, 340)
(15, 357, 710)
(16, 482, 368)
(17, 458, 293)
(18, 562, 494)
(19, 690, 330)
(20, 375, 312)
(21, 537, 500)
(22, 534, 619)
(23, 627, 566)
(24, 722, 579)
(25, 574, 593)
(26, 710, 566)
(27, 518, 386)
(28, 606, 653)
(29, 421, 638)
(30, 456, 718)
(31, 719, 323)
(32, 648, 626)
(33, 367, 348)
(34, 376, 363)
(35, 340, 638)
(36, 701, 520)
(37, 376, 724)
(38, 509, 657)
(39, 459, 621)
(40, 438, 395)
(41, 440, 693)
(42, 355, 549)
(43, 600, 724)
(44, 324, 546)
(45, 478, 290)
(46, 541, 600)
(47, 427, 435)
(48, 314, 501)
(49, 728, 626)
(50, 488, 425)
(51, 547, 534)
(52, 570, 621)
(53, 314, 429)
(54, 347, 325)
(55, 320, 718)
(56, 720, 600)
(57, 711, 394)
(58, 608, 372)
(59, 520, 346)
(60, 632, 630)
(61, 641, 610)
(62, 332, 341)
(63, 317, 487)
(64, 607, 490)
(65, 446, 622)
(66, 508, 551)
(67, 735, 734)
(68,

In [163]:
df.at[0, 'source']

'/mnt/nas/nb_usctsim/sim_001/trial_001'

## SAの性能評価

In [136]:
arr_sa = []
arr_sct = []

for i in range(len(df)):
    
    input_sample = np.load(input_dataset[i])
    output_sample = np.load(output_dataset[i])

    ts = np.log(np.sum(np.sum(input_sample, axis=0), axis=0)+np.power(0.005, 2))
    arr_sa.append(ts[len(ts)//2])
    
    H, W = output_sample.shape
    arr_sct.append(output_sample[H//2, W//2])


In [137]:
plt.clf()
plt.scatter(np.array(arr_sa), np.array(arr_sct))
fig.savefig('./sa_quality.png')